In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Introduction: Business Problem 
In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening a restaurant in New york. And specifically what kind of restaurant should i open. 

Since there are lots of restaurants in New York we will try to detect locations that are not already crowded with restaurants. We are also particularly interested in areas with no restaurants in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to restaurants in the neighborhood, if any
* distance of neighborhood from city center

In [29]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [30]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [31]:
neighborhoods_data = newyork_data['features']

In [32]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [33]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [34]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [35]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [36]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [37]:
Bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
Bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [38]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bronx are 40.8466508, -73.8785937.


In [39]:
# create map of Bronx using latitude and longitude values
map_Bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Bronx_data['Latitude'], Bronx_data['Longitude'], Bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Bronx)  
    
map_Bronx

In [40]:
CLIENT_ID = 'UUZ20CU4EI4X005MRNR3YSPP1ZNMKWBXPDDI35SQKN2GXJ04' # your Foursquare ID
CLIENT_SECRET = 'DDMUGCUSEAFWDFSVOJXUJKDNKAA1Q4YHGZDPLTIUJY3KNVY5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UUZ20CU4EI4X005MRNR3YSPP1ZNMKWBXPDDI35SQKN2GXJ04
CLIENT_SECRET:DDMUGCUSEAFWDFSVOJXUJKDNKAA1Q4YHGZDPLTIUJY3KNVY5


In [41]:
Bronx_data.loc[13, 'Neighborhood']

'University Heights'

In [42]:
neighborhood_latitude = Bronx_data.loc[13, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Bronx_data.loc[13, 'Longitude'] # neighborhood longitude value

neighborhood_name = Bronx_data.loc[13, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of University Heights are 40.85572707719664, -73.9104159619131.


In [43]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=UUZ20CU4EI4X005MRNR3YSPP1ZNMKWBXPDDI35SQKN2GXJ04&client_secret=DDMUGCUSEAFWDFSVOJXUJKDNKAA1Q4YHGZDPLTIUJY3KNVY5&v=20180605&ll=40.85572707719664,-73.9104159619131&radius=500&limit=100'

In [44]:
results = requests.get(url).json()

In [45]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [46]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Accra Resturant,African Restaurant,40.853871,-73.908421
1,Wingstop,Wings Joint,40.854093,-73.907899
2,Domino's Pizza,Pizza Place,40.854541,-73.907037
3,Bravo Supermarkets,Grocery Store,40.854107,-73.914162
4,Food Dynasty,Supermarket,40.853772,-73.909267
5,Chase Bank,Bank,40.854120,-73.907555
6,Papa John's Pizza,Pizza Place,40.852429,-73.908976
7,Liberato,Latin American Restaurant,40.853744,-73.907966
8,Au Bon Pain,Bakery,40.858062,-73.912040
9,Subway,Sandwich Place,40.853923,-73.906948


In [47]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
Bronx_venues = getNearbyVenues(names=Bronx_data['Neighborhood'],
                                   latitudes=Bronx_data['Latitude'],
                                   longitudes=Bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [51]:
print(Bronx_venues.shape)
Bronx_venues.head()

(1203, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [52]:
Bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,31,31,31,31,31,31
Baychester,21,21,21,21,21,21
Bedford Park,39,39,39,39,39,39
Belmont,99,99,99,99,99,99
Bronxdale,11,11,11,11,11,11
Castle Hill,9,9,9,9,9,9
City Island,27,27,27,27,27,27
Claremont Village,16,16,16,16,16,16
Clason Point,10,10,10,10,10,10


In [53]:
print('There are {} uniques categories.'.format(len(Bronx_venues['Venue Category'].unique())))

There are 168 uniques categories.


In [54]:
# one hot encoding
Bronx_onehot = pd.get_dummies(Bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Bronx_onehot['Neighborhood'] = Bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Bronx_onehot.columns[-1]] + list(Bronx_onehot.columns[:-1])
Bronx_onehot = Bronx_onehot[fixed_columns]

Bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nail Salon,Nightclub,Office,Optical Shop,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Shop,Wings Joint,Women's Store
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [55]:
Bronx_onehot.shape

(1203, 169)

In [56]:
Bronx_grouped = Bronx_onehot.groupby('Neighborhood').mean().reset_index()
Bronx_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nail Salon,Nightclub,Office,Optical Shop,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Shop,Wings Joint,Women's Store
0,Allerton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.064516,0.000000,0.000000,0.000000,0.0,0.000000,0.032258,0.000000,0.000000,0.064516,0.032258,0.032258,0.000000,0.032258,0.000000,0.000,0.000,0.032258,0.000000,0.032258,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.032258,0.000000,0.0000,0.032258,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.129032,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.00,0.0,0.000000,0.064516,0.032258,0.00000,0.000000,0.000000,0.000000,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Baychester,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04

In [57]:
Bronx_grouped.shape

(52, 169)

In [58]:
num_top_venues = 5

for hood in Bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Bronx_grouped[Bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                venue  freq
0         Pizza Place  0.13
1       Deli / Bodega  0.06
2                 Spa  0.06
3  Chinese Restaurant  0.06
4              Bakery  0.06


----Baychester----
                  venue  freq
0            Donut Shop  0.10
1    Mexican Restaurant  0.05
2           Supermarket  0.05
3         Shopping Mall  0.05
4  Fast Food Restaurant  0.05


----Bedford Park----
                venue  freq
0  Chinese Restaurant  0.10
1         Pizza Place  0.10
2               Diner  0.10
3  Mexican Restaurant  0.08
4       Deli / Bodega  0.08


----Belmont----
                venue  freq
0  Italian Restaurant  0.20
1         Pizza Place  0.08
2       Deli / Bodega  0.07
3              Bakery  0.05
4  Mexican Restaurant  0.04


----Bronxdale----
                           venue  freq
0                 Breakfast Spot  0.09
1             Italian Restaurant  0.09
2             Spanish Restaurant  0.09
3                    Pizza Place  0.09
4  Paper / Office Supp

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Bronx_grouped['Neighborhood']

for ind in np.arange(Bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Spa,Deli / Bodega,Chinese Restaurant,Bakery,Supermarket,Smoke Shop,Pharmacy,Breakfast Spot,Fast Food Restaurant
1,Baychester,Donut Shop,Spanish Restaurant,Supermarket,Pizza Place,Pet Store,Rental Car Location,Discount Store,Sandwich Place,Shopping Mall,Electronics Store
2,Bedford Park,Chinese Restaurant,Diner,Pizza Place,Deli / Bodega,Mexican Restaurant,Spanish Restaurant,Sandwich Place,Food Truck,Discount Store,Bar
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Mexican Restaurant,Grocery Store,Dessert Shop,Donut Shop,Bank,Fish Market
4,Bronxdale,Mexican Restaurant,Gym,Performing Arts Venue,Chinese Restaurant,Paper / Office Supplies Store,Eastern European Restaurant,Spanish Restaurant,Breakfast Spot,Bank,Pizza Place


In [61]:
# set number of clusters
kclusters = 9

Bronx_grouped_clustering =Bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 7, 4, 1, 4, 1, 5, 4], dtype=int32)

In [62]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Bronx_merged = Bronx_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Bronx_merged = Bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Bronx_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,4,Pharmacy,Ice Cream Shop,Gas Station,Deli / Bodega,Sandwich Place,Dessert Shop,Laundromat,Donut Shop,Fast Food Restaurant,Farmers Market
1,Bronx,Co-op City,40.874294,-73.829939,4,Fast Food Restaurant,Bus Station,Accessories Store,Baseball Field,Grocery Store,Liquor Store,Mattress Store,Discount Store,Park,Pharmacy
2,Bronx,Eastchester,40.887556,-73.827806,4,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Bowling Alley,Food & Drink Shop,Convenience Store,Seafood Restaurant,Donut Shop,Bus Stop
3,Bronx,Fieldston,40.895437,-73.905643,8,Plaza,Wine Shop,River,Bus Station,Dive Bar,Film Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant
4,Bronx,Riverdale,40.890834,-73.912585,0,Park,Playground,Bank,Home Service,Plaza,Food Truck,Bus Station,Gym,Baseball Field,Farmers Market


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bronx_merged['Latitude'], Bronx_merged['Longitude'], Bronx_merged['Neighborhood'], Bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [126]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 0, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Riverdale,Park,Playground,Bank,Home Service,Plaza,Food Truck,Bus Station,Gym,Baseball Field,Farmers Market
17,West Farms,Bus Station,Park,Metro Station,Latin American Restaurant,Playground,Diner,Coffee Shop,Sandwich Place,Donut Shop,Lounge


In [127]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 1, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Kingsbridge,Pizza Place,Bar,Supermarket,Latin American Restaurant,Sandwich Place,Mexican Restaurant,Deli / Bodega,Spanish Restaurant,Bakery,Pharmacy
7,Norwood,Pizza Place,Park,Bank,Pharmacy,Mexican Restaurant,Bus Station,Chinese Restaurant,Pet Store,Restaurant,Fast Food Restaurant
12,Bedford Park,Chinese Restaurant,Diner,Pizza Place,Deli / Bodega,Mexican Restaurant,Spanish Restaurant,Sandwich Place,Food Truck,Discount Store,Bar
13,University Heights,Bakery,Pizza Place,Chinese Restaurant,Laundromat,Burrito Place,Shoe Store,Donut Shop,Pharmacy,Sandwich Place,Fried Chicken Joint
14,Morris Heights,Spanish Restaurant,Pizza Place,Recreation Center,Pharmacy,Deli / Bodega,Grocery Store,Supermarket,Bank,Women's Store,Doctor's Office
16,East Tremont,Pizza Place,Café,Fast Food Restaurant,Breakfast Spot,Supermarket,Shoe Store,Lounge,Donut Shop,Liquor Store,Bank
18,High Bridge,Pharmacy,Pizza Place,Bus Station,Supermarket,Latin American Restaurant,Seafood Restaurant,Sports Club,Bus Stop,Food,Park
19,Melrose,Pizza Place,Pharmacy,Gym / Fitness Center,Sandwich Place,Discount Store,Grocery Store,Clothing Store,Supermarket,Supplement Shop,Diner
20,Mott Haven,Pizza Place,Spanish Restaurant,Donut Shop,Latin American Restaurant,Burger Joint,Storage Facility,Baseball Field,Discount Store,Bakery,Mobile Phone Shop
21,Port Morris,Latin American Restaurant,Furniture / Home Store,Cupcake Shop,Music Venue,Storage Facility,Food Truck,Donut Shop,Brewery,Distillery,Restaurant


In [128]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 2, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Williamsbridge,Nightclub,Soup Place,Bar,Caribbean Restaurant,Women's Store,Donut Shop,Film Studio,Fast Food Restaurant,Farmers Market,Electronics Store


In [129]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 3, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Country Club,Sandwich Place,Playground,Chinese Restaurant,Athletics & Sports,Dive Bar,Film Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant


In [130]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 4, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Ice Cream Shop,Gas Station,Deli / Bodega,Sandwich Place,Dessert Shop,Laundromat,Donut Shop,Fast Food Restaurant,Farmers Market
1,Co-op City,Fast Food Restaurant,Bus Station,Accessories Store,Baseball Field,Grocery Store,Liquor Store,Mattress Store,Discount Store,Park,Pharmacy
2,Eastchester,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Bowling Alley,Food & Drink Shop,Convenience Store,Seafood Restaurant,Donut Shop,Bus Stop
9,Baychester,Donut Shop,Spanish Restaurant,Supermarket,Pizza Place,Pet Store,Rental Car Location,Discount Store,Sandwich Place,Shopping Mall,Electronics Store
10,Pelham Parkway,Pizza Place,Italian Restaurant,Frozen Yogurt Shop,Deli / Bodega,Performing Arts Venue,Coffee Shop,Chinese Restaurant,Sandwich Place,Donut Shop,Smoke Shop
11,City Island,Harbor / Marina,Seafood Restaurant,Thrift / Vintage Store,Boat or Ferry,History Museum,Tapas Restaurant,Park,Pizza Place,Diner,Music Venue
15,Fordham,Mobile Phone Shop,Donut Shop,Shoe Store,Pizza Place,Bank,Supplement Shop,Gym / Fitness Center,Clothing Store,Pharmacy,Spanish Restaurant
22,Longwood,Fast Food Restaurant,Deli / Bodega,Sandwich Place,Latin American Restaurant,Diner,Donut Shop,Grocery Store,Women's Store,Doctor's Office,Film Studio
23,Hunts Point,Waste Facility,Food,Grocery Store,Restaurant,Café,Farmers Market,Spanish Restaurant,Bank,Gourmet Shop,BBQ Joint
24,Morrisania,Discount Store,Bus Station,Fast Food Restaurant,Donut Shop,Grocery Store,Sandwich Place,Fish & Chips Shop,Fish Market,Pizza Place,Bowling Alley


In [131]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 5, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Clason Point,Park,Pool,Bus Stop,Moving Target,Boat or Ferry,South American Restaurant,Grocery Store,Fast Food Restaurant,Farmers Market,Electronics Store


In [132]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 6, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Edenwald,Construction & Landscaping,Fish Market,Supermarket,Gas Station,Grocery Store,Department Store,Dance Studio,Film Studio,Cupcake Shop,Fast Food Restaurant


In [133]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 7, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Woodlawn,Deli / Bodega,Pizza Place,Food & Drink Shop,Playground,Pub,Convenience Store,Auto Workshop,Pharmacy,Food Truck,Bar
27,Throgs Neck,Pizza Place,Sports Bar,Bar,Chinese Restaurant,Coffee Shop,Asian Restaurant,Italian Restaurant,American Restaurant,Deli / Bodega,Baseball Field
33,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Mexican Restaurant,Grocery Store,Dessert Shop,Donut Shop,Bank,Fish Market
38,Edgewater Park,Italian Restaurant,Deli / Bodega,Coffee Shop,Pizza Place,Pub,Bar,American Restaurant,Bookstore,Food & Drink Shop,Japanese Restaurant


In [134]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 8, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Plaza,Wine Shop,River,Bus Station,Dive Bar,Film Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant
